In [3]:
import av
import pandas as pd
from av.video.stream import VideoStream
import numpy as np
import cv2
import os
import catboost

def get_stats_for_file(array_file):
    array_file = np.array(array_file)
    
    minim_mean = array_file.mean() - array_file.min()
    maxim_mean = array_file.max() - array_file.mean()
    std = array_file.std()
    
    diff = np.diff(array_file , n = 1)
    min_max_1 = diff.max() - diff.min()
    std_diff_1 = diff.std()
    
    diff = np.diff(array_file , n = 2)
    min_max_2 = diff.max() - diff.min()
    std_diff_2 = diff.std()
    
    diff = np.diff(array_file , n = 3)
    min_max_3 = diff.max() - diff.min()
    std_diff_3 = diff.std()
    
    return [minim_mean , maxim_mean  ,  std,  min_max_1 ,  std_diff_1 , min_max_2 ,  std_diff_2 ,min_max_3 ,  std_diff_3]

def get_pixel_point(image):
    image = image[0:100,:]
    return image[:,:,0].mean()*0.299 + image[:,:,1].mean()*0.587 + image[:,:,2].mean()*0.114

def get_array(path):
    files = [video for video in os.listdir(path) if ".avi" in video and "._" not in video]
    data = []
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([15, 15, 30])

    kernel = np.ones((5,5),'int')
    for file in (files):
        array_1 = []
        array_2 = []
        array_3 = []
        input_container = av.open(path + file)
        input_packets = input_container.demux()
        for packet in input_packets:
            frames = packet.decode()
            for raw_frame in frames:
                x = raw_frame.to_rgb().to_nd_array()
                
                array_1.append(x[:,:,0].mean()*0.299  + x[:,:,1].mean()*0.587 + x[:,:,2].mean()*0.114)
                array_2.append(get_pixel_point(x))
                
                frame_hsv = cv2.cvtColor(x, cv2.COLOR_RGB2HSV)
                black_mask = cv2.inRange(frame_hsv, lower_black, upper_black)
                black_mask = cv2.dilate(black_mask,kernel)
                array_3.append((black_mask==255).sum()/black_mask.size)
        data.append([file , array_1 , array_2 , array_3])
    return data

def generate_fich(path):
    fich = []
    data =  get_array(path)
    file = data[0]
    for x in data[1:]:
        fich.append(get_stats_for_file(x))
    fich = np.hstack(fich)
    return pd.DataFrame([data , fich] , dtype=float)

def get_fich(data):
    fich = []
    for x in data:
        fich.append(np.hstack([x[0] , get_stats_for_file(x[1]) , get_stats_for_file(x[2]) , get_stats_for_file(x[3])]))
    return pd.DataFrame(fich)

In [4]:
path = 'tipa_valid/'
test = get_array(path)
test_fich = get_fich(test)

model_bridge = catboost.CatBoostClassifier().load_model('model_bridge.cat')
model_screen_wipers = catboost.CatBoostClassifier().load_model('model_screen_wipers.cat')

files_test = [video for video in os.listdir(path) if ".avi" in video and "._" not in video]
porog_bridge = 0.5
porog_screen_wipers = 0.5

In [11]:
sub_screen_wipers = pd.DataFrame(files_test)
sub_screen_wipers['res'] = model_screen_wipers.predict_proba(test_fich.drop([0] , axis=1))[:,1]
sub_screen_wipers['res'] = sub_screen_wipers['res'].apply(lambda x: '000010' if x >= porog_screen_wipers  else '000000')
sub_screen_wipers.to_csv('5.txt' , sep = ' ' , header=None , index=None)